In [1]:
#wrongly used the submission example file for the modeling, going to start things clean over again
#setting up libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRFClassifier
from sklearn.model_selection import train_test_split

In [2]:
#load data
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [109]:
def data_preprocess (source_df):
    
    #first we drop the unwanted columns
    source_df_droped = source_df.drop(['PassengerId', 'Name','Ticket','Cabin'], axis=1)

    
    #then we impute Age
    age_imputer = SimpleImputer()
    source_df_droped_justAge = source_df_droped[['Age']]
    imputed_Age = pd.DataFrame(age_imputer.fit_transform(source_df_droped_justAge))

    #impute removes column name and index, adding it back
    imputed_Age.columns = source_df_droped_justAge.columns
    imputed_Age.index = source_df_droped.index

    #adding imputed data to the source df
    source_df_imputed = source_df_droped.drop('Age', axis=1).join(imputed_Age)
    
    #lesson learned, I should impute data before droping rows, and deleting rows is not a good idea, let's impute with most common values
    source_df_imputed["Embarked"] = source_df_imputed["Embarked"].fillna(source_df_imputed["Embarked"].value_counts().index[0])
    source_df_imputed["Fare"] = source_df_imputed["Fare"].fillna(source_df_imputed["Fare"].value_counts().index[0])
    
    #next we apply One-Hot
    OH_en = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_source_df_imputed_col = OH_en.fit_transform(source_df_imputed[['Sex','Embarked']])
    OH_source_df_imputed_col = pd.DataFrame(OH_source_df_imputed_col)
    
    #alining index 
    OH_source_df_imputed_col.index = source_df_imputed[['Sex','Embarked']].index
    #alining columns
    OH_source_df_imputed_col.columns = OH_en.get_feature_names(['Sex','Embarked'])
    processed_df = source_df_imputed.drop(['Sex','Embarked'], axis=1).join(OH_source_df_imputed_col)

    
    return processed_df

In [110]:
def kv_feature (train_set, test_set,k_value):
    f_cols = train_set.columns.drop('Survived')
    #we have 10 features, let's keep five to avoid overfitting
    selector = SelectKBest(f_classif, k=k_value)
    X_new = selector.fit_transform(train_set[f_cols], train_set['Survived'])
    
    #transforming back to the original df format
    selected_features = pd.DataFrame(selector.inverse_transform(X_new), 
                                 index=train_set.index, 
                                 columns=f_cols)
    selected_columns = selected_features.columns[selected_features.var() != 0]
    
    return train_set[selected_columns], train_set['Survived'], test_set[selected_columns], test_set['Survived']

In [111]:
def L_one_feature (train_set, test_set):
    f_cols = train_set.columns.drop('Survived')
    X, y = train_set[f_cols], train_set['Survived']

    # Setting up the model
    logistic = LogisticRegression(C=1, penalty="l1", solver='liblinear', random_state=7).fit(X, y)
    model = SelectFromModel(logistic, prefit=True)
    
    #applying the model
    X_new = model.transform(X)
    
    #transforming back to the original df format
    selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=X.index,
                                 columns=X.columns)
    selected_columns = selected_features.columns[selected_features.var() != 0]
    
    return train_set[selected_columns], train_set['Survived'], test_set[selected_columns], test_set['Survived']

In [112]:
def acc_cal (pred_data, result):
    comp = pred_data == result
    #need to convert to list to use count(True)
    num_r = comp.tolist().count(True)
    acc_p = num_r/len(comp.tolist())
    return acc_p

In [113]:
def dt (X_train, y_train, X_test, y_test):
    dt_model = DecisionTreeClassifier(random_state=1)
    dt_model.fit(X_train, y_train)
    
    score = acc_cal(dt_model.predict(X_test), y_test)
    return score

In [114]:
#define the Random Forest Model
def rf (X_train, y_train, X_test, y_test):
    forest_model = RandomForestClassifier(n_estimators=100, random_state=1)
    forest_model.fit(X_train, y_train)
    pred = forest_model.predict(X_test)
    score = acc_cal(pred, y_test)
    return score

In [115]:
def xgb (X_train, y_train, X_test, y_test):
    xgb_model = XGBRFClassifier(n_estimators=10000)
    xgb_model.fit(X_train, y_train)
    pred = xgb_model.predict(X_test)
    score = acc_cal(pred, y_test)
    return score
    

In [116]:
def xgb_tuned (X_train, y_train, X_test, y_test):
    xgb_model = XGBRFClassifier(n_estimators=10000, learning_rate=0.05)
    xgb_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], 
             verbose=False)
    pred = xgb_model.predict(X_test)
    score = acc_cal(pred, y_test)
    return score

In [117]:
def run_models(X_train, y_train, X_test, y_test):

    #run decision tree model
    score_dt = dt(X_train, y_train, X_test, y_test)
    
    #run the Random Forest Model
    score_rf = rf(X_train, y_train, X_test, y_test)
    
    #run the XGB tuned model
    score_xgb = xgb_tuned(X_train, y_train, X_test, y_test)
    
    
    print(score_dt)
    print(score_rf)
    print(score_xgb)

In [118]:
train_dataset, val_dataset = train_test_split(train_data, train_size=0.8, test_size=0.2)

In [119]:
train_clean, val_clean = data_preprocess(train_dataset), data_preprocess(val_dataset)

In [120]:
train_clean.isnull().sum()

Survived      0
Pclass        0
SibSp         0
Parch         0
Fare          0
Age           0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [121]:
train_clean.head()

,Survived,Pclass,SibSp,Parch,Fare,Age,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
419,0,3,0,2,24.15,10.00000,1.0,0.0,0.0,0.0,1.0
116,0,3,0,0,7.75,70.50000,0.0,1.0,0.0,1.0,0.0
120,0,2,2,0,73.50,21.00000,0.0,1.0,0.0,0.0,1.0
10,1,3,1,1,16.70,4.00000,1.0,0.0,0.0,0.0,1.0
121,0,3,0,0,8.05,29.77331,0.0,1.0,0.0,0.0,1.0


In [122]:
X_kv_train, y_kv_train, X_kv_test, y_kv_test = kv_feature(train_clean, val_clean, 4)
run_models(X_kv_train, y_kv_train, X_kv_test, y_kv_test)

0.7932960893854749
0.776536312849162
0.776536312849162


In [123]:
X_L_one_train, y_L_one_train, X_L_one_test, y_L_one_test = L_one_feature(train_clean, val_clean)
run_models(X_L_one_train, y_L_one_train, X_L_one_test, y_L_one_test)

0.7094972067039106
0.7877094972067039
0.8044692737430168


In [124]:
run_models(train_clean.drop('Survived', axis = 1), train_clean.Survived, val_clean.drop('Survived', axis = 1), val_clean.Survived)

0.7150837988826816
0.7877094972067039
0.8044692737430168


In [4]:
pd.pivot_table(train_data, index = 'Survived', columns = 'Pclass', aggfunc='count')

Age          Cabin        Embarked          Fare  ... PassengerId  \
Pclass      1   2    3     1   2  3        1   2    3    1  ...           3   
Survived                                                    ...               
0          64  90  270    59   3  6       80  97  372   80  ...         372   
1         122  83   85   117  13  6      134  87  119  136  ...         119   

          Sex          SibSp          Ticket           
Pclass      1   2    3     1   2    3      1   2    3  
Survived                                               
0          80  97  372    80  97  372     80  97  372  
1         136  87  119   136  87  119    136  87  119  

[2 rows x 30 columns]